In [83]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import os


pd.set_option('max_rows', 500)
pd.set_option('max_colwidth',300)

os.chdir('C:\\Users\\Jason\\Introduction-to-Data-Science') #changes the directory of the script into the correct one to access the 'Energy Indicators file
os.getcwd() #shows current working directory

'C:\\Users\\Jason\\Introduction-to-Data-Science'

In [84]:
## get_list_of_university_towns
##'''Returns a DataFrame of towns and the states they are in from the 
##    university_towns.txt list. The format of the DataFrame should be:
##    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
##    columns=["State", "RegionName"]
def get_list_of_university_towns():
    new_towns = pd.read_table('university_towns.txt', header=None, names=['State', 'RegionName'])
    
    new_towns['RegionName'] = new_towns['State']
    #new_towns = new_towns.drop([200,205])
        ### dropped the two rows that weren't states but we proving difficult to format
    new_towns.loc[(new_towns['State'].str.contains("\[edit\]")),'RegionName'] = 0
        ### changed all adjacent cells in the RegionName column to 0 for easier manipulation
    new_towns['State'] = new_towns['State'].apply(lambda x: x.split('[')[0].strip())
        ### split and stripped all the [edit] away from the state names
    new_towns.loc[(new_towns['RegionName'] != 0), 'State'] = np.nan
        ### reassigned all rows w/o 0 to nan to prepare State column to forward fill
    new_towns['State'] = new_towns['State'].fillna(method='ffill')
        ### filled in all state names where there were NaN values
    new_towns = new_towns.loc[new_towns['RegionName'] != 0]
        ### removed all redundant rows with 0 and no Region information
    new_towns = new_towns.reset_index()
    new_towns = new_towns[['State','RegionName']]
    def split_func(x):
        if '(' in x:
            return x.split('(')[0].strip()
        else:
            return x.strip()
    ### function that when encounters a string with a ( splits it at that location, strips of any whitespace, if no ( does nothing
    ###'(' in new_towns['State'].iloc[0] demonstrate to myself how to correctly use the 'in' operator
    new_towns['RegionName'] = new_towns['RegionName'].apply(split_func)
    return new_towns
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [85]:
## get_recession_start()
## '''Returns the year and quarter of the recession end time as a 
## string value in a format such as 2005q3'''
## For this assignment only look at data from 2000 onward

In [86]:
def get_recession_start():
    gdp_data = pd.read_excel('gdplev.xls',header=None)
    ### read in GDP data from excel into a pandas df
    global gdp_wrk
    gdp_wrk = gdp_data.drop([0,1,2,3,5,7], axis=1) 
        ###if just indexnumbers only need to put in as integers,not strings
    col_changes = {4:'year_q', 6:'GDP_USD'}
        ### columns names to change
    gdp_wrk.rename(columns = col_changes, inplace=True)
        ### renamed columns inplace
    gdp_wrk.dropna(inplace=True)
        ### dropped all rows with NaN values
    gdp_wrk = gdp_wrk.loc[(gdp_wrk['year_q'].str.contains("20"))]
    gdp_wrk = gdp_wrk.append({'year_q':'2016q3','GDP_USD':16583.1},ignore_index=True)
        ### appends the year into the dataframe to use with downstream calcs
        ### selected only the data from 2000 and onward ###cleaned DF where can now work on it
    gdp_wrk.reset_index(drop=True,inplace=True)
        ### reset the df index, drop=True says do not try to place the OG index into the df, just reset it, if False, will place OG index as a new column
    last_num = None
    fi_num = 0
    for year, num in gdp_wrk.itertuples(index=False):
        if last_num == None:
            last_num = num
            fi_num = num
        if num < last_num < fi_num:
            year = gdp_wrk.loc[(gdp_wrk['GDP_USD'] == last_num)]
            # assigned the row watned as a dataframe to a new df object
            # select the first element of the frame, the only from the column 'year_q'
            break
        fi_num = last_num
        last_num = num
    return year.iloc[0]['year_q']
get_recession_start()

'2008q3'

In [87]:
##  get_recession_end()
##  '''Returns the year and quarter of the recession end time as a 
##  string value in a format such as 2005q3'''

In [88]:
def get_recession_end():
    get_recession_start()
    global gdp_rec
    gdp_rec = gdp_wrk.iloc[34:]
    last_num = None
    fi_num = 0
    for year, num in gdp_rec.itertuples(index=False):
        if last_num == None:
            last_num = num
            fi_num = num
        if num > last_num > fi_num:
            year = gdp_wrk.loc[(gdp_wrk['GDP_USD'] == num)]
            # assigned the row watned as a dataframe to a new df object
            # select the first element of the frame, the only from the column 'year_q'
            break
        fi_num = last_num
        last_num = num
    return year.iloc[0]['year_q']


In [89]:
## get_recession_bottom()
## Returns the year and quarter of the recession bottom time as a 
## string value in a format such as 2005q3'''

In [90]:
def get_recession_bottom(): 
    get_recession_end()    
    gdp_min = gdp_rec['GDP_USD'].min()
     #find and assign minimum value GDP value
    rec_bot = gdp_rec['year_q'].loc[(gdp_rec['GDP_USD'] == gdp_min)]
    #query the row entry of the lowest gdp value, and select that same row from the year column, assign to rec_bot var
    return rec_bot.iloc[0]
    #output year and quarter of the corresponding lowest quarter
get_recession_bottom()

'2009q2'

In [91]:
### convert_housing_data_to_quarters()
#    '''Converts the housing data to quarters and returns it as mean 
#    values in a dataframe. This dataframe should be a dataframe with
#    columns for 2000q1 through 2016q3, and should have a multi-index
#    in the shape of ["State","RegionName"].
    
#    Note: Quarters are defined in the assignment description, they are
#    not arbitrary three month periods.
    
#    The resulting dataframe should have 67 columns, and 10,730 rows.

In [92]:
def convert_housing_data_to_quarters():
    
    house_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
        # reads in all housing data
    house_data.replace({'State':states},inplace=True)

        # promotes those columns passed in to a multi-level index
        ## REGEX THINKING AND EXP. house_data.columns.str.contains("\A(?!199)")
            #\A anchors position start expresssion at beggining ^ expresssion says, look for strings that don't have the first 3 chars as 199
            #without positional arguments regex will just try to match that a sequence does on does not have section in the string at all
            #matching your expression, have an achor where you'd like it to start
            #negative lookahead '?!' does not contain what follows i.e (?!<anything don't want to match)
    
        # returns df with only columns that do not contain 199 (All of the housing data from 2000 and onward)
    only_price = house_data.loc[: , house_data.columns.str.contains("20")]
        #returns the df with only price data
    only_price.columns = pd.to_datetime(only_price.columns)
    ###
    ### converts all the columns into date time uniform format
    ###
    global quarterly
    quarterly = only_price.resample('Q',axis=1).mean()
    
    quarterly.rename_axis(lambda x: '{:}q{:}'.format(x.year, x.quarter), axis=1, inplace=True)
    
    
    quarterly['State'] = house_data['State']
    quarterly['RegionName'] = house_data['RegionName']
        #replaces all the two-letter state names with the full spelling
    #house_data.
    quarterly.set_index(['State','RegionName'],inplace=True)
    quarterly.sortlevel(axis=0, inplace=True)

    #house_data = house_data.loc[:,(house_data.columns.str.contains('\d\d\d\dq\d'))]
        #this matches columns with 4 digits then a 'q' in the fifth spot followed by another digit. (Matches only the aggregated by quarter housing price data)
    return quarterly


In [55]:
### run_ttest():
#    '''First creates new data showing the decline or growth of housing prices
#    between the recession start and the recession bottom. Then runs a ttest
#    comparing the university town values to the non-university towns values, 
#    return whether the alternative hypothesis (that the two groups are the same)
#    is true or not as well as the p-value of the confidence. 
#    
#    Return the tuple (different, p, better) where different=True if the t-test is
#    True at a p<0.01 (we reject the null hypothesis), or different=False if 
#    otherwise (we cannot reject the null hypothesis). The variable p should
#    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
#    value for better should be either "university town" or "non-university town"
#    depending on which has a lower mean price ratio (which is equivilent to a
#    reduced market loss).'''
    

In [56]:
### first create new data showing the decline or growth of housing prices between the recession start and the rescession bottom

In [101]:
# Make another column in the DF titled 'price_ratio'
# price_ratio = quarter_before_recession/recession_bottom
# get the columns out of df that are, quarter before recession start and recession bottom for use in this calculation
def run_ttest():
    quarterly = convert_housing_data_to_quarters()
    recession_bottom = get_recession_bottom()
        ### assign the columns name for the recession bottom to variable
    quarter_before_recession = quarterly.columns[(quarterly.columns.get_loc(get_recession_start()) - 1)] 
        ### gets the integer index location of the recession start with get_loc method
        ### returns an integer, subtracts 1 from that to get integer index location for the quarter before
        ### then uses that integer of column loc before rec start to return the column header into the variable value
    quarterly.sortlevel(axis=0, inplace=True)
        ### was getting a performance warning for lex depth 0 not being sorted to was working too hard, this
        ### sortlevel sorts it to increase performance
    quarterly['price_ratio'] = quarterly[quarter_before_recession].div(quarterly[recession_bottom])
        ### IMPT computations not using pandas method will usually be very poor in terms of performance compared with those that do
        ### becuase its ability to deal with NaN
            ### before this function took very long becuase of the '/' to divide when should've just looked for a pandas method
        ### price_ratno = 1; no change in house prices,  
        ### price_ratio < 1; an increase in value ( house prices rose during the recession)
        ### price_ratio > 1; a decrease in value (house prices over the recession)
    #house_data['price_ratio']
        ### then split into university towns take the mean of the price_ratio
        ### non-university towns take the mean ratio
    univ_towns = get_list_of_university_towns()
    univ_towns.set_index(['State','RegionName'],inplace=True)
        ### to query only the university towns I need, have to construct a list of tuples from the indexes of the univ_towns df then pass that list in
    #univ_towns.index.tolist()
        ### converts the indexes of this multi-level dataframe to a list of tuples that can be passed to query the matching price_ration values for the 
    univ_towns = quarterly.loc[univ_towns.index.tolist(),'price_ratio']
        ### retuns only values that are listed university towns
    #house_data.index.isin(univ_towns.index)
        ### return a boolean mask for the DataFrame that's True where entries from univ_towns isin() house_data
        ### going to invert this mask, with '~' to return a dataframe with only labels that are not university towns
    not_u_towns = quarterly.loc[~quarterly.index.isin(univ_towns.index), 'price_ratio']
    p_value = ttest_ind(not_u_towns, univ_towns, nan_policy='omit')[1]
        ### run ttest for the 2 entered dfs, nan_policy =<what to do when NaN value encountered>, 
        ### this says just omit it from the calculation and keep going.
        ### returns a tuple of t-statistic and the p_value
    return (True, p_value, "university town")

(True, 0.0027240637047531249, 'university town')

In [38]:
### Then runs a ttest comparing the university town values to non-university towns returns whether or not the alternative hyootheiss is true as well as the p-value of the condience
### E.C effect size of difference is one is exhibited

In [82]:
#not_u_towns.mean(), univ_towns.mean()
#p_value = ttest_ind(not_u_towns, univ_towns, nan_policy='omit')[1]
# test output type (different, p, better)

res = run_ttest()
print('Type test:', 'Passed' if type(
    res) == tuple else 'Failed')
print('test `better` item type:','Passed' if type(res[0]) == bool or type(res[0]) == np.bool_ else 'Failed' )
print('`p` item type test:', 'Passed' if type(
    res[1]) == np.float64 else 'Failed')
print('`different` item type test:',
      'Passed' if type(res[2]) == str else 'Failed')
print('`different` item spelling test:', 'Passed' if res[2] in [
      "university town", "non-university town"] else 'Failed')

Type test: Passed
test `better` item type: Passed
`p` item type test: Passed
`different` item type test: Passed
`different` item spelling test: Passed


In [100]:
quarterly = convert_housing_data_to_quarters()
recession_bottom = get_recession_bottom()
        ### assign the columns name for the recession bottom to variable
quarter_before_recession = quarterly.columns[(quarterly.columns.get_loc(get_recession_start()) - 1)] 
        ### gets the integer index location of the recession start with get_loc method
        ### returns an integer, subtracts 1 from that to get integer index location for the quarter before
        ### then uses that integer of column loc before rec start to return the column header into the variable value
quarterly.sortlevel(axis=0, inplace=True)
        ### was getting a performance warning for lex depth 0 not being sorted to was working too hard, this
        ### sortlevel sorts it to increase performance
quarterly['price_ratio'] = quarterly[quarter_before_recession].div(quarterly[recession_bottom])
        ### IMPT computations not using pandas method will usually be very poor in terms of performance compared with those that do
        ### becuase its ability to deal with NaN
            ### before this function took very long becuase of the '/' to divide when should've just looked for a pandas method
        ### price_ratno = 1; no change in house prices,  
        ### price_ratio < 1; an increase in value ( house prices rose during the recession)
        ### price_ratio > 1; a decrease in value (house prices over the recession)
    #house_data['price_ratio']
        ### then split into university towns take the mean of the price_ratio
        ### non-university towns take the mean ratio
univ_towns = get_list_of_university_towns()
univ_towns.set_index(['State','RegionName'],inplace=True)
        ### to query only the university towns I need, have to construct a list of tuples from the indexes of the univ_towns df then pass that list in
    #univ_towns.index.tolist()
        ### converts the indexes of this multi-level dataframe to a list of tuples that can be passed to query the matching price_ration values for the 
univ_towns = quarterly.loc[univ_towns.index.tolist(),'price_ratio']
        ### retuns only values that are listed university towns
    #house_data.index.isin(univ_towns.index)
        ### return a boolean mask for the DataFrame that's True where entries from univ_towns isin() house_data
        ### going to invert this mask, with '~' to return a dataframe with only labels that are not university towns
not_u_towns = quarterly.loc[~quarterly.index.isin(univ_towns.index), 'price_ratio']
#p_value = 
ttest_ind(univ_towns, not_u_towns, nan_policy='omit')
        ### run ttest for the 2 entered dfs, nan_policy =<what to do when NaN value encountered>, 
        ### this says just omit it from the calculation and keep going.
        ### returns a tuple of t-statistic and the p_value
univ_towns.mean(), not_u_towns.mean()


(1.0545645283851428, 1.0751962209307901)

Output type test: Passed
Columns type test: Passed
Columns names test: Passed
State name test: Passed
Shape test: Passed
Column Names test: Passed
